# addresses / egid

In [ ]:
# Import required libraries
import requests
import json
import logging
from typing import List
from tqdm.notebook import tqdm
import pandas as pd
import sqlite3
from datetime import datetime


def get_addresses_and_egids(url: str, db_path: str = "addresses.db") -> None:
    """
    Retrieve all unique address-EGID pairs from the API and save to SQLite database.

    Args:
        url (str): The API endpoint URL.
        db_path (str): Path to the SQLite database file.
    """
    # Create database connection
    conn = sqlite3.connect(db_path)

    # Create table if it doesn't exist
    conn.execute(
        """
        CREATE TABLE IF NOT EXISTS addresses (
            egid INTEGER,
            address TEXT,
            updated_at TIMESTAMP,
            PRIMARY KEY (egid, address)
        )
    """
    )

    offset = 0
    chunk_size = 1000
    total_saved = 0

    # Make an initial request to get the total count
    params = {
        "where": "1=1",
        "outFields": "adresse,egid",
        "returnGeometry": "false",
        "f": "json",
        "resultRecordCount": 1,
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        total_count = data.get("properties", {}).get("total", 0)

        # Create progress bar
        pbar = tqdm(total=total_count, desc="Fetching data")

        while True:
            params = {
                "where": "1=1",
                "outFields": "adresse,egid",
                "returnGeometry": "false",
                "f": "json",
                "resultOffset": offset,
                "resultRecordCount": chunk_size,
            }

            try:
                response = requests.get(url, params=params)
                response.raise_for_status()
                data = response.json()

                if "features" not in data or not data["features"]:
                    break

                # Prepare data for database
                now = datetime.now().isoformat()
                records = [
                    (
                        feature["attributes"]["egid"],
                        feature["attributes"]["adresse"],
                        now,
                    )
                    for feature in data["features"]
                    if feature["attributes"]["adresse"]
                    and feature["attributes"]["egid"]
                ]

                # Insert data into database
                conn.executemany(
                    """
                    INSERT OR REPLACE INTO addresses (egid, address, updated_at)
                    VALUES (?, ?, ?)
                    """,
                    records,
                )
                conn.commit()

                total_saved += len(records)

                # Update progress bar
                records_fetched = len(data["features"])
                pbar.update(records_fetched)

                if records_fetched < chunk_size:
                    break

                offset += chunk_size

            except requests.exceptions.RequestException as e:
                print(f"Error fetching data at offset {offset}: {e}")
                break
            except json.JSONDecodeError:
                print(f"JSON decode error at offset {offset}")
                break

        pbar.close()

        # Display summary from database
        cursor = conn.execute("SELECT COUNT(*) FROM addresses")
        total_records = cursor.fetchone()[0]
        print(f"\nTotal unique address-EGID pairs in database: {total_records}")

        print("\nFirst few entries:")
        df = pd.read_sql_query(
            """
            SELECT egid, address, updated_at 
            FROM addresses 
            ORDER BY egid, address 
            LIMIT 5
            """,
            conn,
        )
        display(df)

    except Exception as e:
        print(f"Error during execution: {e}")
    finally:
        conn.close()


def query_database(db_path: str = "addresses.db") -> None:
    """
    Display some basic statistics about the database.

    Args:
        db_path (str): Path to the SQLite database file.
    """
    conn = sqlite3.connect(db_path)
    try:
        # Get total count
        cursor = conn.execute("SELECT COUNT(*) FROM addresses")
        total = cursor.fetchone()[0]
        print(f"Total records: {total}")

        # Get some example queries
        print("\nExample queries you can run:")
        print("\n1. Get all addresses for a specific EGID:")
        print("SELECT * FROM addresses WHERE egid = X")

        print("\n2. Search for addresses containing a specific string:")
        print("SELECT * FROM addresses WHERE address LIKE '%search_term%'")

        print("\n3. Get the latest updated records:")
        print("SELECT * FROM addresses ORDER BY updated_at DESC LIMIT 10")

    finally:
        conn.close()




In [11]:
FIELDS = "*"
URL_INDICE_MOYENNES_3_ANS = "https://vector.sitg.ge.ch/arcgis/rest/services/Hosted/SCANE_INDICE_MOYENNES_3_ANS/FeatureServer/0/query"

# Example usage in notebook
db_path = "addresses.db"

# Get addresses and EGIDs and save to database
print("Retrieving addresses and EGIDs...")
get_addresses_and_egids(URL_INDICE_MOYENNES_3_ANS, db_path)

# Show database info and example queries
print("\nDatabase Information:")
query_database(db_path)

Retrieving addresses and EGIDs...


Fetching data: 0it [00:00, ?it/s]


Total unique address-EGID pairs in database: 19983

First few entries:


,egid,address,updated_at
0,1000016,Route d'Hermance 261,2024-12-10T16:18:20.296353
1,1000070,Route d'Hermance 263,2024-12-10T16:18:09.133901
2,1000116,Route d'Hermance 289,2024-12-10T16:18:17.475018
3,1000135,Route d'Hermance 317,2024-12-10T16:18:10.597721
4,1000144,Route d'Hermance 342A,2024-12-10T16:18:13.402714



Database Information:
Total records: 19983

Example queries you can run:

1. Get all addresses for a specific EGID:
SELECT * FROM addresses WHERE egid = X

2. Search for addresses containing a specific string:
SELECT * FROM addresses WHERE address LIKE '%search_term%'

3. Get the latest updated records:
SELECT * FROM addresses ORDER BY updated_at DESC LIMIT 10


In [19]:
conn = sqlite3.connect(db_path)
df = pd.read_sql_query("SELECT * FROM addresses WHERE egid = 1000135", conn)
display(df["address"][0])

"Route d'Hermance 317"